In [4]:
pip list

Package                   Version
------------------------- --------------
anyio                     4.8.0
appdirs                   1.4.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
astunparse                1.6.3
async-lru                 2.0.4
attrs                     25.1.0
autograd                  1.7.0
autoray                   0.7.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
cachetools                5.5.1
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.12
decorator                 5.1.1
defusedxml                0.7.1
diastatic-malt            2.15.2
dill                      0.3.9
executing                 2.2.0
fastjsonschema  

In [7]:
dev = qml.device("default.qubit", wires=[0,1])

def my_first_circuit(theta):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.RZ(theta, wires=0)
    
    return qml.probs(wires=[0,1])

my_first_qnode = qml.QNode(my_first_circuit, dev)

In [2]:
#!/usr/bin/env python

import numpy as np

import pyopencl as cl


rng = np.random.default_rng()
a_np = rng.random(50000000, dtype=np.float64)
b_np = rng.random(50000000, dtype=np.float64)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
knl = prg.sum  # Use this Kernel object for repeated calls
knl(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
error_np = res_np - (a_np + b_np)
print(f"Error:\n{error_np}")
print(f"Norm: {np.linalg.norm(error_np):.16e}")
assert np.allclose(res_np, a_np + b_np)

Error:
[ 29.39741415 103.26138516  75.11582189 ...  -0.88421027  -1.63256968
  -0.90677622]
Norm: 6.3729269518948987e+05


AssertionError: 